In [128]:
import json
import requests
import collections
import urllib.parse
import base64
import hmac
import hashlib
import urllib.request

In [129]:
def mouserSearch(part_number, quantity=1):

    print('Mouser Searching for '+part_number+'...')
    api_key = "82675baf-9a58-4d5a-af3f-e3bbcf486560"
    url = f"https://api.mouser.com/api/v1.0/search/partnumber?apiKey={api_key}"
    body = {
        'SearchByPartRequest': {
            'mouserPartNumber': part_number,
            'partSearchOptions': f'None&qty={quantity}',
        }
    }
    headers = {'Content-Type': 'application/json'}

    response = requests.post(url=url, data=json.dumps(body), headers=headers)
    if response.status_code == 200:
        data = response.json()
        #print(data)
        if 'SearchResults' in data:
            parts = data['SearchResults'].get('Parts', [])
            if len(parts) == 0:
                print("NA\n")
            
            for part in parts:
                price_breaks = part.get('PriceBreaks', [])
                if(len(price_breaks)>0):
                    if(quantity > 0 and quantity < price_breaks[0]['Quantity']):
                        print(f"Minimum Order Quantity is {price_breaks[0]['Quantity']}\n")
                    elif(quantity >= price_breaks[-1]['Quantity']):
                        print(float(price_breaks[-1]['Price'][1:])*quantity, price_breaks[-1]['Currency'], '\n')
                    else:
                        for i in range(len(price_breaks)-1):
                            if quantity >= price_breaks[i]['Quantity'] and quantity<price_breaks[i+1]['Quantity']:
                                print(float(price_breaks[i]['Price'][1:])*quantity, price_breaks[i]['Currency'], '\n')
        else:
            print("NA\n")
    else:
        raise Exception('Mouser API returned: {}'.format(response.status_code)+' - {}'.format(response.reason))

In [130]:
def rutronikSearch(part_number, quantity=1):
    
    print('Rutronik Searching for '+part_number+'...')
    api_key = "cc6qyfg2yfis"
    url = f"https://www.rutronik24.com/api/pricerequest/?apikey={api_key}&sku={part_number}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        #print(data)
        if 'error' in data:
            print("NA\n")
        else:
            price_breaks = data['pricebreaks']
            if(quantity > 0 and quantity < data['moq']):
                print(f"Minimum Order Quantity is {data['moq']}\n")
            elif(quantity >= price_breaks[-1]['quantity']):
                print(float(price_breaks[-1]['price'])*quantity, data['currency'], '\n')
            else:
                for i in range(len(price_breaks)-1):
                    if quantity >= price_breaks[i]['quantity'] and quantity<price_breaks[i+1]['quantity']:
                        print(float(price_breaks[i]['price'])*quantity, data['currency'], '\n')
    else:
        raise Exception('Rutronik API returned: {}'.format(response.status_code)+' - {}'.format(response.reason))

In [131]:
def tmeSearch(part_number, quantity=1):
    # /product/product_import_tme/
    print('TME Searching for '+part_number+'...')

    token = 'c850b76479a46e253c40383eb8907071dc3136d3de975d0fe7'
    app_secret = b'fd5a595a6eda52c936f3'

    params = {
        'SymbolList[0]' : part_number,
        'Country': 'PL',
        'Currency': 'PLN',
        'Language': 'PL',
        'Token' : token,
    }
    params = collections.OrderedDict(sorted(params.items()))

    url = 'https://api.tme.eu/Products/GetPrices.json'

    encoded_params = urllib.parse.urlencode(params, '').encode()
    signature_base = 'POST' + '&' + urllib.parse.quote(url, '') + '&' + urllib.parse.quote(encoded_params, '')

    api_signature = base64.b64encode(hmac.new(app_secret, signature_base.encode(), hashlib.sha1).digest()).rstrip()
    params['ApiSignature'] = api_signature

    http_header = {
        "Content-type": "application/x-www-form-urlencoded",
    }

    # creating HTTP request
    req = urllib.request.Request(url, urllib.parse.urlencode(params).encode(), http_header)

    # submitting request
    try:
        with urllib.request.urlopen(req) as res:
            if res.status == 200:
                response = res.read().decode()
                data = json.loads(response)
                #print(data)
                if data is not None:
                    parts = data['Data'].get('ProductList', [])
                    if len(parts) == 0:
                        print("NA\n")

                    for part in parts:
                        price_breaks = part.get('PriceList', [])
                        if(quantity > 0 and quantity < price_breaks[0]['Amount']):
                            print(f"Minimum Order Quantity is {price_breaks[0]['Amount']}\n")
                        elif(quantity >= price_breaks[-1]['Amount']):
                            print(price_breaks[-1]['PriceValue']*quantity, data['Data']['Currency'], '\n')
                        else:
                            for i in range(len(price_breaks)-1):
                                if quantity >= price_breaks[i]['Amount'] and quantity<price_breaks[i+1]['Amount']:
                                    print(price_breaks[i]['PriceValue']*quantity, data['Data']['Currency'], '\n')
                else:
                    print("NA\n")       
            else:
                print(f"TME API returned: {res.status} - {res.reason}\n")
    except urllib.error.HTTPError as e:
        print(f"TME API returned: {e.code} - {e.reason}\n")

In [132]:
# Example usage
partNumbers = ["CC0603KRX7R8BB105", "ICMCU22246", "ICMCU22250", "KKK19097", "NE555D", "1N4007-DC"]
quantity = 1000

for p in partNumbers:
    mouserSearch(p, quantity)
    rutronikSearch(p, quantity)
    tmeSearch(p, quantity)

Mouser Searching for CC0603KRX7R8BB105...
4730.0 INR 

Rutronik Searching for CC0603KRX7R8BB105...
NA

TME Searching for CC0603KRX7R8BB105...
44.4 USD 

Mouser Searching for ICMCU22246...
NA

Rutronik Searching for ICMCU22246...
10710.0 EUR 

TME Searching for ICMCU22246...
TME API returned: 406 - Not Acceptable

Mouser Searching for ICMCU22250...
NA

Rutronik Searching for ICMCU22250...
8000.0 EUR 

TME Searching for ICMCU22250...
TME API returned: 406 - Not Acceptable

Mouser Searching for KKK19097...
NA

Rutronik Searching for KKK19097...
Minimum Order Quantity is 15000

TME Searching for KKK19097...
TME API returned: 406 - Not Acceptable

Mouser Searching for NE555D...
33370.0 INR 

10710.0 INR 

Rutronik Searching for NE555D...
NA

TME Searching for NE555D...
498.0 USD 

Mouser Searching for 1N4007-DC...
NA

Rutronik Searching for 1N4007-DC...
NA

TME Searching for 1N4007-DC...
25.0 USD 

